In [22]:
import datasets
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.pyplot as plt
import numpy as np
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from utils import remove_extra_brackets
from transformers import AutoTokenizer, AutoModelForMaskedLM
from torch.nn import Linear, BCEWithLogitsLoss
import torch

In [2]:
# Load multiple CSV files
df = datasets.load_dataset('csv', data_files={
    'train': '../data/train.csv',
    'test': '../data/test.csv'
})

In [4]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/roberta-base")
model = AutoModelForMaskedLM.from_pretrained("FacebookAI/roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [5]:
model

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNor

In [7]:
model.lm_head.decoder = Linear(in_features=768, out_features=3, bias=False)

In [19]:
encoded_text = tokenizer(df["train"]["response_a"][:2], return_tensors="pt", truncation=True, padding="longest")["input_ids"]

In [20]:
encoded_text.shape

torch.Size([2, 512])

In [23]:
with torch.no_grad():
    outputs = model(encoded_text)

In [27]:
outputs.logits.shape

torch.Size([2, 512, 3])